In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt
from streamlit_keplergl import keplergl_static

In [2]:
# Define the folder path
folderpath = "../Citibike_Project/Data/Prepared_data"
# Load the DataFrame from the pickle file
df = pd.read_pickle(os.path.join(folderpath, 'cleaned_nyc_bike_weather_data.pkl'))

In [3]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,date,avgTemp,trip_duration,month,season,value,bike_rides_daily,log_trip_duration
0,BFD29218AB271154,Electric Bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802116,-73.968178,...,-73.945923,Member,2022-01-21,-6.0,528,1,Winter,1,33598,6.270988
1,096AE4C21473D1D6,Electric Bike,2022-01-21 20:32:55.968,2022-01-21 20:35:20.227,Riverside Dr & W 91 St,7524.16,Riverside Dr & W 82 St,7388.10,40.793137,-73.977005,...,-73.981277,Member,2022-01-21,-6.0,144,1,Winter,1,33598,4.976734
2,B3996D7CAEDF7B92,Classic Bike,2022-01-21 09:14:04.856,2022-01-21 09:18:53.000,Henry St & Atlantic Ave,4531.05,Clinton St & Tillary St,4748.07,40.690891,-73.996124,...,-73.991417,Member,2022-01-21,-6.0,288,1,Winter,1,33598,5.666427
3,83AE874442FFF355,Electric Bike,2022-01-21 17:30:27.244,2022-01-21 17:36:00.300,Cleveland Pl & Spring St,5492.05,Rivington St & Ridge St,5406.02,40.722103,-73.997246,...,-73.983299,Member,2022-01-21,-6.0,333,1,Winter,1,33598,5.811141
4,8B1950B6ED5BD50A,Electric Bike,2022-01-21 10:12:57.242,2022-01-21 10:31:37.016,1 Ave & E 62 St,6753.08,Allen St & Rivington St,5414.06,40.761227,-73.960938,...,-73.989975,Member,2022-01-21,-6.0,1119,1,Winter,1,33598,7.021084


In [4]:
df.shape

(29838166, 21)

In [38]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp', 'trip_duration', 'month', 'season',
       'value', 'bike_rides_daily', 'log_trip_duration'],
      dtype='object')

In [53]:
# Convert from category to object (string)
df['start_station_name'] = df['start_station_name'].astype(str)
df['end_station_name'] = df['end_station_name'].astype(str)

In [54]:
sub_df= df.drop(columns = {'ride_id', 'started_at', 'ended_at', 'start_station_id',
                         'end_station_id', 'start_lat', 'start_lng', 'end_lat',
                         'end_lng', 'member_casual', 'month', 'log_trip_duration'})

In [55]:
sub_df.head()

,rideable_type,start_station_name,end_station_name,date,avgTemp,trip_duration,season,value,bike_rides_daily
0,Electric Bike,West End Ave & W 107 St,Mt Morris Park W & W 120 St,2022-01-21,-6.0,528,Winter,1,33598
1,Electric Bike,Riverside Dr & W 91 St,Riverside Dr & W 82 St,2022-01-21,-6.0,144,Winter,1,33598
2,Classic Bike,Henry St & Atlantic Ave,Clinton St & Tillary St,2022-01-21,-6.0,288,Winter,1,33598
3,Electric Bike,Cleveland Pl & Spring St,Rivington St & Ridge St,2022-01-21,-6.0,333,Winter,1,33598
4,Electric Bike,1 Ave & E 62 St,Allen St & Rivington St,2022-01-21,-6.0,1119,Winter,1,33598


In [56]:
df_temps = sub_df[['date', 'avgTemp']]
df_temps = df_temps.groupby('date', as_index = False).agg(pd.Series.mode)

In [57]:
df_daily_rides = sub_df[['date', 'value']]
df_daily_rides = df_daily_rides.groupby('date', as_index = False).sum('value')
df_daily_rides = df_daily_rides.rename(columns = {'value': 'Daily Rides'})

In [58]:
df_classic_daily_rides = sub_df[['date', 'rideable_type', 'value']]
df_classic_daily_rides = df_classic_daily_rides.groupby(['date', 'rideable_type'], as_index = False).sum('value')
df_classic_daily_rides = df_classic_daily_rides.loc[df_classic_daily_rides['rideable_type'] == 'Classic Bike']
df_classic_daily_rides = df_classic_daily_rides.rename(columns = {'value': 'Daily Classic Rides'})

In [59]:
line_chart_data = df_temps.merge(df_daily_rides, on = 'date', how = 'left')


In [60]:
line_chart_data = line_chart_data.merge(df_classic_daily_rides[['date', 'Daily Classic Rides']], 
                                        on = 'date', how = 'left')

In [61]:
line_chart_data = line_chart_data.rename(columns = {'date' : 'Date', 'avgTemp' : 'Average Temperature'})
line_chart_data.head()


,Date,Average Temperature,Daily Rides,Daily Classic Rides
0,2022-01-01,11.6,20428,11399
1,2022-01-02,11.4,43009,26857
2,2022-01-03,1.4,33189,19583
3,2022-01-04,-2.7,36842,22476
4,2022-01-05,3.2,34230,20676


In [82]:
#EXPORT DATA
line_chart_data.to_csv(os.path.join(folderpath, 'DB_line_chart_data.csv'), index=True)

In [62]:
df_bar = sub_df[['start_station_name', 'end_station_name', 'rideable_type', 'season', 'value']]

In [63]:
df_bar.head()

,start_station_name,end_station_name,rideable_type,season,value
0,West End Ave & W 107 St,Mt Morris Park W & W 120 St,Electric Bike,Winter,1
1,Riverside Dr & W 91 St,Riverside Dr & W 82 St,Electric Bike,Winter,1
2,Henry St & Atlantic Ave,Clinton St & Tillary St,Classic Bike,Winter,1
3,Cleveland Pl & Spring St,Rivington St & Ridge St,Electric Bike,Winter,1
4,1 Ave & E 62 St,Allen St & Rivington St,Electric Bike,Winter,1


In [64]:
df_bar_start = df_bar.groupby('start_station_name', as_index=False).sum('value')
df_top_start = df_bar_start.nlargest(20, 'value')
top_start_list = df_top_start['start_station_name'].tolist()

In [65]:
top_start_data = df_bar.loc[df_bar['start_station_name'].isin(top_start_list)]
grouped_start = top_start_data.groupby(['start_station_name', 'rideable_type', 'season'], 
                                       as_index=False).sum('value')
grouped_start = grouped_start.rename(columns = {'value' : 'Total'})

In [66]:
grouped_start.head()

,start_station_name,rideable_type,season,Total
0,1 Ave & E 68 St,Classic Bike,Fall,14736
1,1 Ave & E 68 St,Classic Bike,Spring,15016
2,1 Ave & E 68 St,Classic Bike,Summer,19145
3,1 Ave & E 68 St,Classic Bike,Winter,7940
4,1 Ave & E 68 St,Electric Bike,Fall,14289


In [67]:
df_bar_end = df_bar.groupby('end_station_name', as_index=False).sum('value')
df_top_end = df_bar_end.nlargest(20, 'value')
top_end_list = df_top_end['end_station_name'].tolist()

In [68]:
top_end_data = df_bar.loc[df_bar['end_station_name'].isin(top_end_list)]
grouped_end = top_end_data.groupby(['end_station_name', 'rideable_type', 'season'], 
                                       as_index=False).sum('value')
grouped_end = grouped_end.rename(columns = {'value' : 'Total'})

In [69]:
grouped_start = grouped_start.merge(df_top_start[['start_station_name', 'value']], on='start_station_name', how='left')
grouped_end = grouped_end.merge(df_top_end[['end_station_name', 'value']], on='end_station_name', how='left')

In [70]:
grouped_start = grouped_start.rename(columns = {'value' : 'Grand Total'})
grouped_end = grouped_end.rename(columns = {'value' : 'Grand Total'})

In [71]:
grouped_start.head()

,start_station_name,rideable_type,season,Total,Grand Total
0,1 Ave & E 68 St,Classic Bike,Fall,14736,104856
1,1 Ave & E 68 St,Classic Bike,Spring,15016,104856
2,1 Ave & E 68 St,Classic Bike,Summer,19145,104856
3,1 Ave & E 68 St,Classic Bike,Winter,7940,104856
4,1 Ave & E 68 St,Electric Bike,Fall,14289,104856


In [84]:
# Export data
grouped_start.to_csv(os.path.join(folderpath, 'DB_bar_chart_start.csv'), index=True)

In [72]:
grouped_end.head()

,end_station_name,rideable_type,season,Total,Grand Total
0,1 Ave & E 68 St,Classic Bike,Fall,14813,105121
1,1 Ave & E 68 St,Classic Bike,Spring,15068,105121
2,1 Ave & E 68 St,Classic Bike,Summer,19085,105121
3,1 Ave & E 68 St,Classic Bike,Winter,7786,105121
4,1 Ave & E 68 St,Electric Bike,Fall,14425,105121


In [85]:
#Export Data
grouped_end.to_csv(os.path.join(folderpath, 'DB_bar_chart_end.csv'), index=True)

In [73]:
df_payment = df[['member_casual', 'value']]

In [74]:
df_payment_pie = df_payment.groupby('member_casual').sum('value')

In [75]:
df_payment_pie

,value
member_casual,
Casual,6580665
Member,23257501


In [88]:
#Export Data
df_payment_pie.to_csv(os.path.join(folderpath, 'DB_pie_payment.csv'), index=True)

In [77]:
df_duration = df[['trip_duration', 'value', 'member_casual', 'season']]

In [78]:
df_duration.head()

,trip_duration,value,member_casual,season
0,528,1,Member,Winter
1,144,1,Member,Winter
2,288,1,Member,Winter
3,333,1,Member,Winter
4,1119,1,Member,Winter


In [79]:
np.random.seed(32)
red = np.random.rand(len(df_duration)) <= 0.975

In [80]:
small = df_duration[~red]

In [81]:
small.shape

(745384, 4)

In [86]:
#Export Data
small.to_csv(os.path.join(folderpath, 'DB_hist_duration.csv'), index=True)